In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import random
import time
from utils import *
from models import *
from attention import *

import torch
import torch.optim as optim
from torch.utils import data
from sklearn.model_selection import train_test_split
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# Download the file
get_file()
path_to_file = 'spa-eng/spa.txt'

======== THE DATASET IS READY ========


In [4]:
# Try experimenting with the size of that dataset
num_examples = 118960
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset(path_to_file, num_examples)

# Calculate max_length of the target tensors
max_length_targ, max_length_inp = max_length(target_tensor), max_length(input_tensor)

The number of line in the dataset is 118964


In [5]:
# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.1)

# Show length
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

107064 107064 11896 11896


In [6]:
print ("Input Language; index to word mapping")
convert(inp_lang, input_tensor_train[0])
print ()
print ("Target Language; index to word mapping")
convert(targ_lang, target_tensor_train[0])

Input Language; index to word mapping
1 ----> <start>
6 ----> el
985 ----> caballo
1568 ----> salto
10 ----> la
273 ----> cerca
3 ----> .
2 ----> <end>

Target Language; index to word mapping
1 ----> <start>
5 ----> the
897 ----> horse
1462 ----> jumped
182 ----> over
5 ----> the
1097 ----> fence
3 ----> .
2 ----> <end>


In [7]:
BATCH_SIZE = 64
steps_per_epoch = len(input_tensor_train)// BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1

In [8]:
# transform to torch tensor
tensor_x = torch.Tensor(input_tensor_train).long() 
tensor_y = torch.Tensor(target_tensor_train).long()
# create your datset
my_dataset = data.TensorDataset(tensor_x,tensor_y) 
# create your dataloader
my_dataloader = data.DataLoader(my_dataset,
                        batch_size=BATCH_SIZE,
                        shuffle=True,
                        drop_last=True,
                        num_workers=4)   

In [9]:
example_input_batch, example_target_batch = next(iter(my_dataloader))

In [10]:
example_input_batch.shape

torch.Size([64, 42])

In [11]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

Encoder output shape: (batch size, sequence length, units) torch.Size([64, 42, 1024])
Encoder Hidden state shape: (batch size, units) torch.Size([1, 64, 1024])


In [12]:
attention_layer = BahdanauAttention(10, 1024)
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

print("Attention result shape: (batch size, units) {}".format(attention_result.shape))
print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))

Attention result shape: (batch size, units) torch.Size([64, 1024])
Attention weights shape: (batch_size, sequence_length, 1) torch.Size([64, 42, 1])


In [13]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE, 1024)

sample_decoder_output, _, _ = decoder(torch.randint(1, 20, (BATCH_SIZE, 1)),
                                      sample_hidden, sample_output)

print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))

Decoder output shape: (batch_size, vocab size) torch.Size([64, 12928])


In [14]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE).to(device)
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE, 1024).to(device)

encoder_optimizer = optim.Adam(encoder.parameters())
decoder_optimizer = optim.Adam(decoder.parameters())

criterion = nn.NLLLoss()

In [15]:
# Train the model
EPOCHS = 10

for epoch in range(EPOCHS):
    start = time.time()

    total_loss = 0

    for (batch, (inp, targ)) in enumerate(my_dataloader):
        inp, targ = inp.to(device), targ.to(device)
        batch_loss = train_step(inp, targ, encoder, decoder,
                                encoder_optimizer, decoder_optimizer,
                                criterion, device, BATCH_SIZE, targ_lang)
        
        total_loss += batch_loss

        if batch % 100 == 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1, batch, batch_loss))
            
    # saving (checkpoint) the model every 2 epochs
    if (epoch + 1) % 2 == 0:
        
        pass

    print('Epoch {} Loss {:.4f}'.format(epoch + 1, total_loss / steps_per_epoch))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 9.2608
Epoch 1 Batch 100 Loss 1.2064
Epoch 1 Batch 200 Loss 1.0059
Epoch 1 Batch 300 Loss 0.9665
Epoch 1 Batch 400 Loss 0.8580
Epoch 1 Batch 500 Loss 1.1753
Epoch 1 Batch 600 Loss 0.6670
Epoch 1 Batch 700 Loss 1.2707
Epoch 1 Batch 800 Loss 0.8587
Epoch 1 Batch 900 Loss 0.8064
Epoch 1 Batch 1000 Loss 0.5194
Epoch 1 Batch 1100 Loss 0.6535
Epoch 1 Batch 1200 Loss 0.5530
Epoch 1 Batch 1300 Loss 0.7081
Epoch 1 Batch 1400 Loss 0.4992
Epoch 1 Batch 1500 Loss 0.7700
Epoch 1 Batch 1600 Loss 0.5741
Epoch 1 Loss 0.9015
Time taken for 1 epoch 1096.3403565883636 sec

Epoch 2 Batch 0 Loss 0.7643
Epoch 2 Batch 100 Loss 0.3729
Epoch 2 Batch 200 Loss 0.6865
Epoch 2 Batch 300 Loss 0.4449
Epoch 2 Batch 400 Loss 0.6956
Epoch 2 Batch 500 Loss 0.6835
Epoch 2 Batch 600 Loss 0.3482
Epoch 2 Batch 700 Loss 0.6420
Epoch 2 Batch 800 Loss 0.5980
Epoch 2 Batch 900 Loss 0.6265
Epoch 2 Batch 1000 Loss 0.4531
Epoch 2 Batch 1100 Loss 0.3546
Epoch 2 Batch 1200 Loss 0.6791
Epoch 2 Batch 1300 Loss 0.3

In [16]:
translate(u'trata de averiguarlo .', max_length_targ, max_length_inp, encoder, decoder, inp_lang, targ_lang, device)

Input: <start> trata de averiguarlo . <end>
Predicted translation: try to figure it out . <end> 


In [17]:
translate(u'¿ todavia estan en casa ?', max_length_targ, max_length_inp, encoder, decoder, inp_lang, targ_lang, device)

Input: <start> ¿ todavia estan en casa ? <end>
Predicted translation: are you still at home ? <end> 


In [18]:
translate(u'esta es mi vida .', max_length_targ, max_length_inp, encoder, decoder, inp_lang, targ_lang, device)

Input: <start> esta es mi vida . <end>
Predicted translation: this is my life . <end> 


In [19]:
translate(u'hace mucho frio aqui .', max_length_targ, max_length_inp, encoder, decoder, inp_lang, targ_lang, device)

Input: <start> hace mucho frio aqui . <end>
Predicted translation: it s very cold here cold here . <end> 


# FIN